In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.42B.300d'
max_size = 64
min_freq = 5
batch_size = 64

In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, max_size, min_freq, batch_size)

In [4]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.333333333333333'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
for i in range(i):
    if len(AttractiveData.test_data[i].Headline) >= max_len:
        max_len = len(AttractiveData.test_data[i].Headline)
max_len

31

## Start to train

In [7]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_name = 'AttractiveNet'
num_workers = 10
input_dim = len(AttractiveData.TEXT.vocab)
embedding_dim = 300

category_dim = len(AttractiveData.CATEGORIES_LABEL.vocab)
category_output_dim = 16
hidden_dim = 256
output_dim = 1
log_steps = 10
epochs = 200
lr = {
    'transformer_encoder': 1e-5,
    'embedding': 1e-5,
    'linear': 1e-4
}
num_layers = 4
nhead = 4
dropout = 0.1
pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1519, 300])


In [8]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [9]:
AttractiveTrainer = AttractiveTrainer(save_name, log_steps, epochs, lr, timestr, AttractiveData.device, AttractiveData.trainloader, input_dim, category_dim, category_output_dim, embedding_dim, hidden_dim, output_dim, pretrained_embeddings, dropout, num_layers, nhead)

In [10]:
AttractiveTrainer.model

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1519, 300, padding_idx=1)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn)

In [11]:
AttractiveTrainer.train()

100%|| 32/32 [00:00<00:00, 106.43it/s]

EP_train | avg_loss: 0.5446687769144773 |
Epoch 888
EP: train | lr: {'transformer_encoder': 1e-05, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 102.56it/s]

EP_train | avg_loss: 0.5450941622257233 |
Epoch 889
EP: train | lr: {'transformer_encoder': 1e-05, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 106.86it/s]

EP_train | avg_loss: 0.5448642922565341 |
Epoch 890
EP: train | lr: {'transformer_encoder': 1e-05, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 106.39it/s]

EP_train | avg_loss: 0.5446487432345748 |
Epoch 891
EP: train | lr: {'transformer_encoder': 1e-05, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 103.80it/s]

EP_train | avg_loss: 0.5471216924488544 |
Epoch 892
EP: train | lr: {'transformer_encoder': 1e-05, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 102.81it/s]

EP_train | avg_loss: 0.5460886945948005 |
Epoch 893
EP: train | lr: {'

## for classification, not better

In [14]:
# from sklearn.metrics import mean_squared_error
# a = AttractiveTrainer.train_predict
# AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
# correct = 0
# pred_list = []
# true_list = []
# for i in range(len(a)):
#     pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
#     pred_list.append(float(pred))
#     true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
#     true_list.append(float(true))
# mean_squared_error(true_list, pred_list)
# # true_list

0.5601443355119825

## Below is testing

In [12]:
from transformermodel import TransformerModel
PATH = './model/AttractiveNet_20201030-104806_0.543.1000'
load_model = TransformerModel(nhead, input_dim, category_dim, category_output_dim, embedding_dim, hidden_dim, output_dim, dropout, num_layers).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1519, 300, padding_idx=1)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn)

In [13]:
def predict_attractive(sentence, category):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)
    tensor_category = tensor_category

    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [14]:
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv('transformers.csv', index=False)

In [15]:
train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
for each_test in test_category:
    if each_test not in train_category:
        print(each_test)
print()
for each_train in train_category:
    if each_train not in test_category:
        print(each_train)


golf
beauty


In [17]:
train_category, test_category

(['travel',
  'health',
  'femail',
  'sport',
  'gardening',
  'sciencetech',
  'news',
  'food',
  'football',
  'travelnews',
  'cricket',
  'golf',
  'books',
  'rugbyunion',
  'home',
  'boxing',
  'tennis',
  'concussion',
  'othersports',
  'beauty',
  'formulaone',
  'racing'],
 ['health',
  'femail',
  'sciencetech',
  'travel',
  'news',
  'football',
  'food',
  'living',
  'books',
  'boxing',
  'rugbyunion',
  'othersports',
  'formulaone',
  'cricket',
  'us',
  'tennis',
  'sport',
  'middleeast',
  'racing'])

## Below just for fun guess

In [17]:
a = AttractiveData.df_test['ID'].to_list()

In [13]:
from sklearn.metrics import mean_squared_error

In [9]:
mean_squared_error(a, b)
# Training all 3.0 got mse = 0.5545

0.5545343137254902

In [18]:
# only for fun
import random
guess_list = [2.3333333333333335, 3.3333333333333335, 3.6666666666666665, 2.6666666666666665]
b = []
for i in range(len(a)):
    b.append(random.choice(guess_list))

In [19]:
AttractiveData.df_test['Label'] = b

In [20]:
AttractiveData.df_test[['ID', 'Label']].to_csv('all_3.csv', index=False)